In [8]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [9]:
img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 16

In [10]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ridiculously_small'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
# test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
# test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [62]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = (descrs.astype(np.float64) / 255)
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [63]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (35, 31, 31)   descrs (35, 129)
patches (25, 31, 31)   descrs (25, 129)


In [64]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [65]:
x_validation.shape

(25, 31, 31, 1)

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [66]:
y_validation[0]

array([-0.18039216, -0.16862745, -0.16862745, -0.18823529, -0.14509804,
       -0.17254902, -0.10588235, -0.01960784, -0.16862745, -0.14509804,
       -0.09803922, -0.17647059, -0.14509804,  0.00392157,  0.0745098 ,
       -0.06666667, -0.19215686, -0.19607843, -0.16862745, -0.14509804,
        0.16470588,  0.29019608, -0.10980392, -0.15294118, -0.17254902,
       -0.19607843, -0.19607843, -0.19607843, -0.16470588, -0.10588235,
       -0.11764706, -0.02352941, -0.18823529, -0.02745098,  0.29019608,
       -0.14117647, -0.19607843, -0.2       , -0.19607843, -0.18039216,
       -0.13333333, -0.0745098 ,  0.29019608,  0.01176471, -0.17254902,
       -0.18823529, -0.19607843, -0.18431373,  0.03529412, -0.1254902 ,
        0.05098039, -0.08235294, -0.01176471, -0.10588235,  0.03529412,
        0.08627451, -0.18823529, -0.2       , -0.2       , -0.2       ,
       -0.16862745,  0.10196078,  0.29019608, -0.01176471, -0.2       ,
       -0.10196078,  0.25098039, -0.09019608, -0.2       , -0.2 

In [67]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="relu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="tanh")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="binary_crossentropy")
#next up: encoder.compile(optimizer='sgd', metrics=['categorical_accuracy'], loss='categorical_crossentropy')


encoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 16)          0   

In [49]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [68]:
model_version = '2.0.0.0.7_encoder_tanhlast_-0.2..0.8_bce_adadelta_ridiculouslysmall'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=500,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0]#,
#                 callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

# import keras.losses
# keras.losses.MSLE_plus_plus = MSLE_plus_plus
# encoder = load_model(base_dir + '/ae' + model_version + '.h5')

Epoch 1/500
35/35 [==============================] - 1s 21ms/step - loss: -1.3535 - val_loss: -1.3650
Epoch 2/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4398 - val_loss: -1.3630
Epoch 3/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4485 - val_loss: -1.3724
Epoch 4/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4449 - val_loss: -1.3427
Epoch 5/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4492 - val_loss: -1.3746
Epoch 6/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4478 - val_loss: -1.3698
Epoch 7/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4541 - val_loss: -1.3836
Epoch 8/500
35/35 [==============================] - 1s 16ms/step - loss: -1.4420 - val_loss: -1.3819
Epoch 9/500
35/35 [==============================] - 1s 15ms/step - loss: -1.4508 - val_loss: -1.3811
Epoch 10/500
35/35 [==============================] - 1s 16ms/step - loss: -1.4482

35/35 [==============================] - 1s 15ms/step - loss: -1.5410 - val_loss: -1.3896
Epoch 81/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5471 - val_loss: -1.3759
Epoch 82/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5418 - val_loss: -1.3874
Epoch 83/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5488 - val_loss: -1.3896
Epoch 84/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5422 - val_loss: -1.3892
Epoch 85/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5484 - val_loss: -1.3864
Epoch 86/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5390 - val_loss: -1.3959
Epoch 87/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5472 - val_loss: -1.3953
Epoch 88/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5417 - val_loss: -1.3956
Epoch 89/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5465 - v

35/35 [==============================] - 1s 15ms/step - loss: -1.5679 - val_loss: -1.3871
Epoch 160/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5616 - val_loss: -1.3910
Epoch 161/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5685 - val_loss: -1.3907
Epoch 162/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5617 - val_loss: -1.3922
Epoch 163/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5683 - val_loss: -1.3926
Epoch 164/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5607 - val_loss: -1.3898
Epoch 165/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5682 - val_loss: -1.3877
Epoch 166/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5616 - val_loss: -1.3899
Epoch 167/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5684 - val_loss: -1.3916
Epoch 168/500
35/35 [==============================] - 1s 15ms/step - loss: -1

Epoch 238/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3873
Epoch 239/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3865
Epoch 240/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3865
Epoch 241/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3872
Epoch 242/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3864
Epoch 243/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3871
Epoch 244/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3868
Epoch 245/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3864
Epoch 246/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5654 - val_loss: -1.3863
Epoch 247/500
35/35 [==============================] - 1s 15ms/s

Epoch 317/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3862
Epoch 318/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3863
Epoch 319/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3869
Epoch 320/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3862
Epoch 321/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3861
Epoch 322/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3864
Epoch 323/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3869
Epoch 324/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3861
Epoch 325/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3861
Epoch 326/500
35/35 [==============================] - 1s 15ms/s

Epoch 396/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3891
Epoch 397/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3881
Epoch 398/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3886
Epoch 399/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3890
Epoch 400/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3884
Epoch 401/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5723 - val_loss: -1.3887
Epoch 402/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3885
Epoch 403/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5723 - val_loss: -1.3890
Epoch 404/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5654 - val_loss: -1.3891
Epoch 405/500
35/35 [==============================] - 1s 15ms/s

Epoch 475/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3876
Epoch 476/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3886
Epoch 477/500
35/35 [==============================] - 1s 16ms/step - loss: -1.5740 - val_loss: -1.3881
Epoch 478/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3887
Epoch 479/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3889
Epoch 480/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3885
Epoch 481/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3886
Epoch 482/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5670 - val_loss: -1.3885
Epoch 483/500
35/35 [==============================] - 1s 15ms/step - loss: -1.5740 - val_loss: -1.3887
Epoch 484/500
35/35 [==============================] - 1s 15ms/s

In [21]:
np.set_printoptions(suppress=True)

In [69]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[-255., -255., -255., -255., -255., -255., -255., -255.,  255.,
        -255., -255., -255., -255., -255., -255., -255.,  255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255.,  255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255.,  255., -255., -255., -255.,
         255., -255., -255., -255., -255., -255., -255., -255.,  255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
        -255., -255., -255., -255.,  255., -255., -255., -255., -255.,
        -255., -255., -255., -255., -255., -255., -255., -255., -255.,
      

In [46]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[ 46.,  23.,  14.,  11.,  20.,  15.,   8.,  20.,  80.,  46.,  20.,
         10.,  14.,  13.,  13.,  29.,  61.,  33.,  20.,  19.,  29.,  22.,
         19.,  27.,  20.,  20.,  11.,  24.,  40.,  22.,  25.,  18.,  42.,
         17.,  16.,  18.,  25.,  20.,  14.,  23.,  97.,  42.,  23.,  15.,
         19.,  16.,  14.,  33.,  85.,  43.,  17.,  22.,  36.,  28.,  27.,
         35.,  28.,  22.,  15.,  36.,  47.,  25.,  23.,  24.,  37.,  18.,
         11.,  12.,  27.,  35.,  28.,  30.,  90.,  28.,  14.,  15.,  23.,
         28.,  23.,  39.,  84.,  43.,  24.,  31.,  37.,  20.,  10.,  28.,
         27.,  23.,  27.,  37.,  42.,  20.,  12.,  21.,  28.,  11.,  13.,
         15.,  28.,  25.,  21.,  30.,  63.,  21.,  17.,  20.,  23.,  21.,
         18.,  42.,  50.,  23.,  19.,  28.,  33.,  19.,  16.,  35.,  25.,
         23.,  18.,  20.,  37.,  17.,  10.,  18., 147.]], dtype=float32)

In [22]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000003.bmp").reshape(1,31,31,1))*255)

array([[  6226.,  -4327.,   -212.,  -5056.,  -7646.,  -6081.,   8609.,
          7355.,   5532.,   6354.,   5112.,   1746.,    469.,   4434.,
          5450.,   1333.,   5431.,  10231.,   6492.,     11.,   3948.,
          3639.,  -1945.,  -1682.,   -183.,  -2503.,   -992.,   -431.,
          4361.,   8124.,   2575.,  -1841.,   3022.,    -79.,   1061.,
         -3145.,   2384.,   -716.,   8723.,   2711.,   7590.,  -2574.,
          2260.,   -314.,   -263.,    203.,    861.,  -2148.,   4120.,
          1577.,   1211.,  -3280.,   2762.,   7455.,   6437.,   1561.,
           170.,  -4488.,   -589.,    465.,   2837.,   2627.,   3180.,
           319.,   1134.,   -845.,  -1492.,  -2196.,  -1676.,  12213.,
          -201.,   5783.,   7160.,   1915.,    378.,   4552.,   4211.,
           557.,   4883.,   4785.,   4315.,  -4458., -12434.,  -8737.,
          5939.,   8673.,   -182.,   -775.,   3257.,  -3857.,  -2258.,
         -3089.,  -1088.,  -2382.,  -1468.,   2597.,  -1078.,  -1202.,
      

In [45]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[ 46.,  23.,  14.,  11.,  20.,  15.,   8.,  20.,  80.,  46.,  20.,
         10.,  14.,  13.,  13.,  29.,  61.,  33.,  20.,  19.,  29.,  22.,
         19.,  27.,  20.,  20.,  11.,  24.,  40.,  22.,  25.,  18.,  42.,
         17.,  16.,  18.,  25.,  20.,  14.,  23.,  97.,  42.,  23.,  15.,
         19.,  16.,  14.,  33.,  85.,  43.,  17.,  22.,  36.,  28.,  27.,
         35.,  28.,  22.,  15.,  36.,  47.,  25.,  23.,  24.,  37.,  18.,
         11.,  12.,  27.,  35.,  28.,  30.,  90.,  28.,  14.,  15.,  23.,
         28.,  23.,  39.,  84.,  43.,  24.,  31.,  37.,  20.,  10.,  28.,
         27.,  23.,  27.,  37.,  42.,  20.,  12.,  21.,  28.,  11.,  13.,
         15.,  28.,  25.,  21.,  30.,  63.,  21.,  17.,  20.,  23.,  21.,
         18.,  42.,  50.,  23.,  19.,  28.,  33.,  19.,  16.,  35.,  25.,
         23.,  18.,  20.,  37.,  17.,  10.,  18., 147.]], dtype=float32)

In [30]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)

In [39]:
y_validation[0]

array([0.01960784, 0.03137255, 0.03137255, 0.01176471, 0.05490196,
       0.02745098, 0.09411765, 0.18039216, 0.03137255, 0.05490196,
       0.10196078, 0.02352941, 0.05490196, 0.20392157, 0.2745098 ,
       0.13333333, 0.00784314, 0.00392157, 0.03137255, 0.05490196,
       0.36470588, 0.49019608, 0.09019608, 0.04705882, 0.02745098,
       0.00392157, 0.00392157, 0.00392157, 0.03529412, 0.09411765,
       0.08235294, 0.17647059, 0.01176471, 0.17254902, 0.49019608,
       0.05882353, 0.00392157, 0.        , 0.00392157, 0.01960784,
       0.06666667, 0.1254902 , 0.49019608, 0.21176471, 0.02745098,
       0.01176471, 0.00392157, 0.01568627, 0.23529412, 0.0745098 ,
       0.25098039, 0.11764706, 0.18823529, 0.09411765, 0.23529412,
       0.28627451, 0.01176471, 0.        , 0.        , 0.        ,
       0.03137255, 0.30196078, 0.49019608, 0.18823529, 0.        ,
       0.09803922, 0.45098039, 0.10980392, 0.        , 0.        ,
       0.        , 0.        , 0.2627451 , 0.22745098, 0.38039

In [13]:
encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_test16/class0/patch000026.png").reshape(1,16,16,3))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [14]:
import tensorflow as tf

print(tf.__version__)

1.14.0


In [19]:
tf.disable_v2_behavior()

W0122 20:00:43.071549 140376228677440 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)